# Exploratory Data Analysis (EDA)

## Project Overview

Customer churn is a critical problem for the banking industry: retaining existing customers is significantly cheaper than acquiring new ones. This project aims to build an end-to-end **machine learning pipeline** to predict customer churn and to identify the key factors that influence a customer's decision to leave the bank.

## EDA Objectives

This notebook covers:
* Dataset overview and data quality checks
* Class balance analysis
* Feature distributions
* Comparison of churned vs non-churned customers
* Initial business insights

sync test
